In [1]:
from IPython.display import FileLink, Image
import pandas as pd
import pandas.rpy.common as com
import brewer2mpl
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import gridspec
from matplotlib import collections
from numpy import nan, min, array
import itertools
from mpltools import style
style.use("ggplot")
%matplotlib inline
%load_ext rpy2.ipython

/opt/anaconda/lib/python2.7/site-packages/pandas/rpy/__init__.py:11: FutureWarning: The pandas.rpy module is deprecated and will be removed in a future version. We refer to external packages like rpy2. 
See here for a guide on how to port your code to rpy2: http://pandas.pydata.org/pandas-docs/stable/r_interface.html
  FutureWarning)
/opt/anaconda/lib/python2.7/site-packages/mpltools/style/__init__.py:42: FutureWarning: 

    The style-sheet functionality in mpltools has been integrated into
    Matplotlib >= 1.4. This module will be removed in a future release.

    Note that style-sheets used by `matplotlib.style` use the standard
    Matplotlib rc-file syntax instead of the INI format used by `mpltools`.
    This mostly means un-quoting strings and changing '=' to ':'.


  """, FutureWarning)


In [2]:
%%R
library(phyloseq)
library(plyr); library(dplyr)

/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘dplyr’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  res = super(Function, self).__call__(*new_args, **new_kwargs)


In [3]:
%%R
physeq = import_biom("../data/otu_table_tax_no211_meta.biom", "../data/ssu-aln.bacteria.mask.tre", parseFunction = parse_taxonomy_greengenes)

In [4]:
%%R
t = tax_table(physeq)
levels(data.frame(t)$Phylum)

 [1] "Acidobacteria"    "Actinobacteria"   "AD3"              "Armatimonadetes" 
 [5] "Bacteroidetes"    "BHI80-139"        "BRC1"             "Chlamydiae"      
 [9] "Chlorobi"         "Chloroflexi"      "Cyanobacteria"    "Elusimicrobia"   
[13] "FBP"              "FCPU426"          "Fibrobacteres"    "Firmicutes"      
[17] "GAL15"            "Gemmatimonadetes" "GN02"             "GN04"            
[21] "Kazan-3B-28"      "MVP-21"           "Nitrospirae"      "NKB19"           
[25] "OD1"              "OP11"             "OP3"              "Planctomycetes"  
[29] "Proteobacteria"   "SBR1093"          "Spirochaetes"     "Tenericutes"     
[33] "[Thermi]"         "TM6"              "TM7"              "Verrucomicrobia" 
[37] "WPS-2"            "WS2"              "WS3"             


In [5]:
#%%R
#TT = data.frame(tax_table(physeq)) %>% add_rownames("OTU")

#read.csv("../data/l2fc.csv") %>%
#    left_join(TT) %>% 
#    write.csv("../data/l2fc_w_tax.csv", row.names = FALSE)

In [6]:
matplotlib.rcParams["axes.facecolor"] = "white"
matplotlib.rcParams["axes.edgecolor"] = "#151515"
matplotlib.rcParams["grid.color"] = "#959595"
matplotlib.rcParams["grid.alpha"] = "0.50"
matplotlib.rcParams["font.size"] = 7.5

In [11]:
def get_l2fc_df(df_fp, taxon, rank):
    df = pd.read_csv(df_fp, sep=",")
    df_subset = df[df[rank]==taxon]
    return df_subset, df_subset["OTU"]

def get_tree_df(physeq_str, OTUs):
    subset_fn_str = "prune_taxa(OTUs, %s)" %physeq_str
    print subset_fn_str
    %Rpush subset_fn_str
    %Rpush OTUs
    %R OTUs = as.character(OTUs)
    %R ps = eval(parse(text=subset_fn_str))
    %R tree_dfs = phyloseq:::tree_layout(phy_tree(ps), ladderize="right")
    %R tax = data.frame(tax_table(ps), stringsAsFactors = FALSE)
    %R tax$OTU = rownames(tax)
    %R tax[is.na(tax)] = 'unassigned'
    %R edge_df = left_join(data.frame(tree_dfs$edgeDT), tax)
    %R edge_df$OTU = factor(edge_df$OTU)
    %R vert_df = data.frame(tree_dfs$vertDT)
    edge_df = com.load_data("edge_df")
    print edge_df
    vert_df = com.load_data("vert_df")
    return edge_df, vert_df

def add_tree_info(tip_df, l2fc_df):
    l2fc_df = l2fc_df.merge(tip_df[["y","OTU"]], 
                            left_on="OTU", 
                            right_on="OTU")
    return l2fc_df

def get_tax_colordict(tip_df, color_rank):
    cats = set(tip_df[color_rank].dropna())
    c_list = []
    for m in ["Set3", "Dark2", "Accent", "Set1"]:
        colors = brewer2mpl.get_map(m, "qualitative", 8).hex_colors
        [c_list.append(c) for c in colors]
    color_dict = dict(itertools.izip(cats, itertools.cycle(c_list)))
    color_dict["unassigned"] = "#505050"
    return color_dict

def get_responders(l2fc_df, amendment):
    min_padj = l2fc_df[["OTU","padj","Amdmt"]].groupby(["Amdmt","OTU"]).min()
    t = min_padj["padj"]
    tmp = t[amendment]
    rsp = tmp[tmp<0.10].index.tolist()
    return rsp
    
def make_tree_fig(taxon, taxon_rank, color_rank, l2fc_fp, physeq_str, lw_tax = 3, leg_spacing = 0.25, 
                  save_fn = None, pt_size = 50):
    l2fc_df, otus = get_l2fc_df(l2fc_fp, taxon, taxon_rank)
    edge_df, vert_df = get_tree_df(physeq_str, otus.tolist())
    tip_df = edge_df[edge_df["OTU"].notnull()]
    l2fc_df = add_tree_info(tip_df, l2fc_df)
    tax_colors = get_tax_colordict(tip_df, color_rank)
    
    days = map(str,[12,82])
    scaleFactor = 0.125
    
    pyom_rspdrs = get_responders(l2fc_df, 1)
    stover_rspdrs = get_responders(l2fc_df, 2)

    df_pivot = l2fc_df.pivot_table(index=["y"], columns=["Amdmt","Day"], values="l2FC")
    figHeight = min(array([df_pivot.shape[0]*scaleFactor, 30]))

    gs = gridspec.GridSpec(nrows=1, ncols=4, width_ratios=[6,2.25,3*.75,3*.75])

    fig = plt.figure()
    fig.set_size_inches((5,figHeight))
    #fig.set_size_inches((8,20))
    fig.subplots_adjust(wspace=0.05)

    cm = brewer2mpl.get_map('RdYlBu', 'diverging', 5, reverse=True).mpl_colormap

    edge_segments = edge_df.apply(lambda row: [(row["xleft"], row["y"]-1), (row["xright"], row["y"]-1)], axis = 1).values
    edge_collection = collections.LineCollection(segments = edge_segments, color="k", linewidths = 1.5)
    
    vert_segments = vert_df.apply(lambda row: [(row["x"], row["vmin"]-1), (row["x"], row["vmax"]-1)], axis = 1).values
    vert_collection = collections.LineCollection(segments = vert_segments, color="k", linewidths = 1.5)

    max_right = tip_df["xright"].max() * 1.05
    buff = 0.025
    verts = tip_df.apply(lambda row: [(row["xright"] + buff, row["y"] - 1.5), 
                                      (row["xright"] + buff, row["y"] - 0.5),
                                      (max_right, row["y"] - 0.5), 
                                      (max_right, row["y"] - 1.5)], axis = 1).values
    rect_collection = collections.PolyCollection(verts, 
                                             facecolors = tip_df.apply(lambda x: tax_colors[x[color_rank]], axis = 1),
                                             alpha = 0.75)
    
    ax1 = fig.add_subplot(gs[0])
    ax1.add_collection(edge_collection)
    ax1.add_collection(vert_collection)
    ax1.add_collection(rect_collection)
    ax1.set_xlim(edge_df["xleft"].min() - edge_df["xright"].max() * 0.05, edge_df["xright"].max() * 1.05)
    ax1.set_ylim(edge_df["y"].min() - edge_df["y"].max() * 0.05, edge_df["y"].max() * 1.05)
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_frame_on(False)
    ax1.grid(False)
    ax1.set_title(taxon, fontdict = {"fontsize": 14})

    ax2 = fig.add_subplot(gs[1], sharey = ax1)
    criterion2 = tip_df["OTU"].map(lambda x: x in pyom_rspdrs)
    criterion3 = tip_df["OTU"].map(lambda x: x in stover_rspdrs)
    pyom_rspndrs_tips = tip_df[criterion2]
    stover_rspndrs_tips = tip_df[criterion3]
    ax2.scatter(x = [1 for i in pyom_rspndrs_tips.y], 
                y = pyom_rspndrs_tips.y -1, 
                s = pt_size, zorder = 100, color = "#377eb8")
    ax2.scatter(x = [2 for i in stover_rspndrs_tips.y], 
                y = stover_rspndrs_tips.y -1, 
                s = pt_size, zorder = 100, color = "#4daf4a")
    ax2.set_frame_on(False)
    ax2.set_xlim([0,3])
    ax2.set_xticks([1,2])
    ax2.set_xticklabels(["PyOM","Stover"], rotation = 90, ha = "left", size = 8)
    ax2.set_yticks([])
    
    vmin = l2fc_df["l2FC"].quantile(0.025)
    vmax = l2fc_df["l2FC"].quantile(0.975)

    ax3 = fig.add_subplot(gs[2], sharey = ax1)
    ax3.set_xticks(range(len(days)))
    ax3.set_xticklabels(days)
    ax3.set_yticklabels([])
    ax3.set_title("PyOM")
    ax3.set_xlabel("Day")
    df = df_pivot[1]
    im = ax3.imshow(df.as_matrix(), interpolation="nearest", aspect="auto", cmap=cm, vmax=vmax, vmin=vmin)
    ax3.grid(False)

    ax4 = fig.add_subplot(gs[3], sharey = ax1)
    ax4.set_xticks(range(len(days)))
    ax4.set_xticklabels(days)
    ax4.set_yticklabels([])
    ax4.set_title("Stover")
    ax4.set_xlabel("Day")
    df = df_pivot[2]
    ax4.imshow(df.as_matrix(), interpolation="nearest", aspect="auto", cmap=cm, vmax=vmax, vmin=vmin)
    ax4.grid(False)

    box = ax4.get_position()
    cax = fig.add_axes([box.xmax + 0.05, box.ymin + 2 * box.height / 3, 0.02, box.height / 3])
    cb = fig.colorbar(im, cax = cax)
    cb.set_label(r"log$_{2}$ Fold Change")

    proxy_artists = [plt.Rectangle((0,0),1,1,fc=tax_colors[i],alpha=0.75) 
                     for i in tip_df.sort("y")[color_rank].unique().tolist()]
    leg = ax1.legend(proxy_artists, 
                     tip_df.sort("y")[color_rank].unique().tolist(),
                     bbox_to_anchor = (0, -leg_spacing),
                     loc = 2, ncol = 2,
                     prop={'size':10,'style':"italic"}, labelspacing = 0.2)
    leg.set_frame_on(False)
    
    if save_fn:
        fig.savefig(save_fn, bbox_extra_artists=(leg, cax), bbox_inches='tight')

In [12]:
make_tree_fig("p__Acidobacteria", "Rank2", "Class", "../data/l2fc_w_tax.csv", "physeq", 
              save_fn = "Gemm_tree.pdf", lw_tax = 1, leg_spacing = 0.03,
              #save_fn = None, lw_tax = 1, leg_spacing = 0.05,
              pt_size = 35)

prune_taxa(OTUs, physeq)


NotImplementedError: Conversion 'ri2py' not defined for objects of type '<class 'rpy2.robjects.methods.RS4'>'